# calculate

In [1]:
import os
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

/home/huqiang/anaconda3/envs/SLTnet/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
pred_root = '../../datasets/snapshot_download_weights/TestHardDataset/Seen/'
img_root = '/memory/yizhenyu/dataset/SUN/data/SUN-SEG/TestHardDataset/Seen/Frame/'
gt_root = '/memory/yizhenyu/dataset/SUN/data/SUN-SEG/TestHardDataset/Seen/GT/'
badsamples = []
badsamples_dice = []
bindex = 0

def cofficent_calculate(preds,gts,threshold=0.5):
    eps = 1e-5
    preds = preds > threshold
    intersection = (preds * gts).sum()
    union =(preds + gts).sum()
    dice = 2 * intersection  / (union + eps)
    iou = intersection/(union - intersection + eps)
    return (dice, iou)

def prepare_data(pred: np.ndarray, gt: np.ndarray) -> tuple:
    if not pred.shape == gt.shape:
        pred = cv2.resize(pred, (gt.shape[1], gt.shape[0]))

    gt = gt/255 if gt.max() >128 else gt
    pred = pred / 255 if pred.max() > 128 else pred
    if pred.max() != pred.min():
        pred = (pred - pred.min()) / (pred.max() - pred.min())
        
    return pred, gt

res = {}
overall_meandice = []
for case in os.listdir(pred_root):
    res[case] = []
    tmp_list = os.listdir(os.path.join(pred_root,case))
    tmp_list.sort(key=lambda name: (
        int(name.split('_a')[1].split('_')[0]),
        int(name.split('_image')[1].split('.png')[
                0])))
    for name in tqdm(tmp_list,ncols=70):
        pred_path = os.path.join(pred_root,case,name)
        gt_path = os.path.join(gt_root,case,name)
        pred = cv2.imread(pred_path,cv2.IMREAD_GRAYSCALE)
        gt = cv2.imread(gt_path,cv2.IMREAD_GRAYSCALE)
        pred,gt = prepare_data(pred,gt)
        dice, iou = cofficent_calculate(pred,gt)
        overall_meandice.append(dice)
        res[case].append(dice)

100%|███████████████████████████████| 236/236 [00:09<00:00, 23.92it/s]


In [8]:
print('overall_meandice: {:.4f}'.format(np.mean(overall_meandice)))
D = np.mean([np.mean(res[case]) for case in res])
print('Meandice on cases: {:.4f}'.format(D))

overall_meandice: 0.8405
Meandice on cases: 0.8383


# calculate and find bad samples

In [ ]:
import os
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

class Detect():
    def __init__(self) :
    
        self.pred_root = '../Polyp-Video-Segmentation/VPS/res/snapshot_download_weights/TestHardDataset/Seen/'
        self.img_root = '/memory/yizhenyu/dataset/SUN/data/SUN-SEG/TestHardDataset/Seen/Frame/'
        self.gt_root = '/memory/yizhenyu/dataset/SUN/data/SUN-SEG/TestHardDataset/Seen/GT/'
        self.badsamples = []
        self.badsamples_dice = []
        self.bindex = 0

    def cofficent_calculate(self,preds,gts):
        eps = 1e-5
        intersection = (preds * gts).sum()
        union =(preds + gts).sum()
        dice = 2 * intersection  / (union + eps)
        iou = intersection/(union - intersection + eps)
        return (dice, iou)
    
    def prepare_data(self,pred: np.ndarray, gt: np.ndarray) -> tuple:
        if not pred.shape == gt.shape:
            pred = cv2.resize(pred, (gt.shape[1], gt.shape[0]))
        gt /= (gt.max() + 1e-8)
        pred = pred / 255 if pred.max() > 128 else pred
        if pred.max() != pred.min():
            pred = (pred - pred.min()) / (pred.max() - pred.min())
            
        return pred, gt

    def find_bad_res(self):
        for case in os.listdir(self.pred_root):
            tmp_list = os.listdir(os.path.join(self.pred_root,case))
            tmp_list.sort(key=lambda name: (
                int(name.split('_a')[1].split('_')[0]),
                int(name.split('_image')[1].split('.png')[
                        0])))
            for name in tqdm(tmp_list,ncols=70):
                pred_path = os.path.join(self.pred_root,case,name)
                gt_path = os.path.join(self.gt_root,case,name)
                pred = cv2.imread(pred_path,cv2.IMREAD_GRAYSCALE)
                gt = cv2.imread(gt_path,cv2.IMREAD_GRAYSCALE)
                pred,gt = self.prepare_data(pred,gt)
                dice, iou = self.cofficent_calculate(pred,gt)
                if dice < 0.5:
                    self.badsamples.append(pred_path)
                    self.badsamples_dice.append(dice)

    def show_badsamplets(self):
        pred_path = self.badsamples[self.bindex]
        self.bindex += 1
        gt_path = pred_path.replace(self.pred_root,self.gt_root)
        img_path = gt_path.replace('GT','Frame').replace('.jpg','.png')

        pred = cv2.imread(pred_path,cv2.IMREAD_GRAYSCALE)
        gt = cv2.imread(gt_path,cv2.IMREAD_GRAYSCALE)
        img = cv2.imread(pred_path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        pred,gt = self.prepare_data(pred,gt)
        print(img_path)
        plt.subplot(131)
        plt.imshow(img)
        plt.subplot(132)
        plt.imshow(gt)
        plt.subplot(133)
        plt.imshow(pred)


In [ ]:
det = Detect()
det.find_bad_res()
bad = det.badsamples
bda_dice = det.badsamples_dice

In [ ]:
from PIL import Image
save_path = '/memory/yizhenyu/dataset/SUN/data/SUN-SEG/TestHardDataset/Seen/keshihua_with_PNS+/'
os.makedirs(save_path,exist_ok=True)
print(len(bad))
for i in range(len(bad)):
    pred_path = bad[i]
    gt_path = pred_path.replace(det.pred_root,det.gt_root)
    img_path = gt_path.replace('GT','Frame').replace('.png','.jpg')

    pred = cv2.imread(pred_path)
    gt = cv2.imread(gt_path)
    img = cv2.imread(img_path)
    # img = cv2.cvtColor(img)
    name = img_path.split('/')[-2] +'_' + img_path.split('/')[-1].replace('.jpg','.png')
    # name = img_path.split('/')[-1].replace('.jpg','.png')
    # print(img_path)
    pred,gt = det.prepare_data(pred,gt)
    h,w,c = map(int,img.shape)
    # plt.subplot(221)
    # plt.subplot(222)
    # plt.imshow(gt)
    # plt.subplot(223)
    # plt.imshow(pred)
    blank = np.ones_like(img)
    dice = det.badsamples_dice[i]
    text = f'dice: {dice:.2f}'
    blank = cv2.putText(blank, text, (200, 200), cv2.FONT_HERSHEY_SIMPLEX,
                                    5.0, (0, 0, 0), 10, cv2.LINE_AA, False)
    img_zhong1 = np.vstack((img/255, pred))
    img_zhong2 = np.vstack((gt, blank))
    # # axis=1：横向合并 axis=0 纵向合并
    img_final = np.concatenate([img_zhong1, img_zhong2], axis=1)

    # plt.imshow(img_final)
    cv2.imwrite(os.path.join(save_path,name),np.uint8(img_final*255))
    # img_final.shape
    # result_image = Image.new("RGB", (w*2, h*2))
    # result_image.paste(img, (0, 0))
    # result_image.paste(gt, (h, 0))
    # result_image.paste(pred, (0, w))
    # result_image = np.array(result_image)
    # plt.imshow(result_image)

In [8]:
pred = np.array([True,True,False])
gt = np.array([1,0,1])
(pred+gt).sum()

4